In [ ]:
import os
import torch
import imageio
import torchvision
from torch.utils.data import Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import imageio.v2 as imageio

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

In [ ]:
# !pip install torchvision

In [ ]:
torch.cuda.is_available()

In [ ]:
raw_data_train = '/Users/aiden/OneDrive/Desktop/Cybersecurity/CIFAR-10/CIFAR-10-images-master/train'
raw_data_test  = '/Users/aiden/OneDrive/Desktop/Cybersecurity/CIFAR-10/CIFAR-10-images-master/test'

In [ ]:
dataset_train = []
labels_train  = []
targets_train = []

In [ ]:
for folder in os.listdir(raw_data_train):
    print(folder)
    for image in os.listdir( os.path.join(raw_data_train, folder)):
        if folder not in labels_train:
            labels_train.append(folder)
        targets_train.append(labels_train.index(folder))
        img_arr = imageio.imread(os.path.join(raw_data_train, folder, image), pilmode="RGB")
        img = torch.from_numpy(img_arr).permute(2, 0, 1).float()
        img /= 255
        dataset_train.append(img)

In [ ]:
len(targets_train)

In [ ]:
dataset_train[3].shape

In [ ]:
data_train = torch.stack(dataset_train)
targets_train = torch.Tensor(targets_train).type(torch.LongTensor)

torch.save((data_train, targets_train, labels_train), "CIFAR10_data")

In [ ]:
data_train.shape

In [ ]:
targets_train.shape

In [ ]:
data_train[4].shape

In [ ]:
targets_train[24000:25000]

In [ ]:
img_tr = data_train[46000]
img_tr

In [ ]:
transform = T.ToPILImage()

In [ ]:
img = transform(img_tr)

In [ ]:
y_train_np = targets_train.numpy()
y_train_np.shape
the_set = np.unique(y_train_np)
the_set

In [ ]:
_ = plt.hist(targets_train.numpy(), bins="auto")
plt.show()

In [ ]:
dataset_test = []
labels_test = []
targets_test = []

In [ ]:
for folder in os.listdir(raw_data_test):
    print(folder)
    for image in os.listdir(os.path.join(raw_data_test, folder)):
        if folder not in labels_test:
            labels_test.append(folder)
        targets_test.append(labels_test.index(folder))
        img_arr = imageio.imread( os.path.join(raw_data_test, folder, image), pilmode="RGB")
        img = torch.from_numpy(img_arr).permute(2, 0, 1).float()
        img /= 255
        dataset_test.append(img)

In [ ]:
data_test = torch.stack(dataset_test)
targets_test = torch.Tensor(targets_test).type(torch.LongTensor)
torch.save((data_test, targets_test, labels_test), "CIFAR10_data_test")

In [ ]:
data_test.shape

In [ ]:
targets_test.shape

In [ ]:
_ = plt.hist( targets_test.numpy() , bins="auto" )
plt.show()

In [ ]:
X_train = data_train
y_train = targets_train

In [ ]:
X_test = data_test
y_test = targets_test

In [ ]:
X_train.shape[0]

In [ ]:
y_train[30000].item()

In [ ]:
X_train[78]

In [ ]:
CIFAR_train_list = [(X_train[i], y_train[i].item()) for i in range(X_train.shape[0])]

In [ ]:
CIFAR_test_list = [(X_test[i], y_test[i].item()) for i in range(X_test.shape[0])]

In [ ]:
batch_size = 16

In [ ]:
train_dl = torch.utils.data.DataLoader(CIFAR_train_list, batch_size=batch_size, shuffle=True)

In [ ]:
test_dl = torch.utils.data.DataLoader(CIFAR_test_list, batch_size=10000, shuffle=True)

In [ ]:
class MLP_net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear( 32*32*3 ,20)
        self.act1    = nn.Sigmoid()
        self.linear2 = nn.Linear(20 , 10)
        self.act2    = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        y_pred = self.act2(x)
        return y_pred

In [ ]:
def training_loop( N_Epochs, model, loss_fn, opt ):
    for epoch in range(N_Epochs):
        for xb, yb in train_dl:
            xb = xb.view((16, -1))
            y_pred = model(xb)
            loss = loss_fn(y_pred, yb)
            opt.zero_grad()
            loss.backward()
            opt.step()
        if epoch % 5 == 0:
            print(epoch, "loss=", loss)

In [ ]:
N_Epochs = 500
learning_rate = 0.001

In [ ]:
model = MLP_net()
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
training_loop(N_Epochs, model, loss_fn, opt)

In [ ]:
def print_metrics_function(y_test, y_pred):
    print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Confusion Matrix:")
    print(confmat)
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('F1-measure: %.3f' % f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

In [ ]:
with torch.no_grad():
    for x_real, y_real in test_dl:
        batch_size = x_real.shape[0]
        x_real = x_real.view((batch_size, -1))
        y_pred = model(x_real)
        vals, indeces = torch.max(y_pred, dim=1)
        preds = indeces
        print_metrics_function(y_real, preds)